## Cleaning up the code

**Inputs**
- parent directory of a S2 image (.SAFE folder)
- `envelope_gdf` (for now I will also include the code to go from buffer to envelope here)

In [70]:
# Imports. To be cleaned as well

import pandas as pd
from matplotlib import pyplot as plt
from shapely.geometry import box
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.coords import BoundingBox
from rasterio.features import geometry_mask
import json
import geopandas as gpd
import numpy as np
import glob
path = r'C:\Users\Kostas\Downloads\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\GRANULE\L2A_T32UNB_A009543_20170420T103454\IMG_DATA\R10m\L2A_T32UNB_20170420T103021_B02_10m.jp2'

### ------------ Buffer to envelope -----------------

In [2]:
# Read buffer file
buffers = gpd.read_file(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\pep725_high_count_days\buffers_day_92.geojson')
# Set crs because by default GeoPandas loads it in 4326, whereas it is actually 32632
buffers.set_crs(32632, inplace=True, allow_override=True)

# Create envelopes for the buffers
envelope_series = buffers.geometry.envelope
envelope_series.rename('envelope_geometry', inplace=True)
envelope_gdf = buffers.merge(envelope_series, left_index=True, right_index=True)
envelope_gdf = envelope_gdf.drop(['geometry'], axis=1).set_geometry('envelope_geometry').rename_geometry('geometry')


## Define functions to be used

In [3]:
"""Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""

def getFeatures(gdf):
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [4]:
"""
Function to clip a GeoDataFrame to the extent of the raster. Can be used for each input image.

It should be changed for something better for better operation. Instead of clipping it should just check if the 
polygon is inside the raster's bounds and just parse it for later processing4

"""

def clipGdfToRasterExtent (raster, envelope_gdf):
    with rasterio.open(path, driver='JP2OpenJPEG') as src:
        # Get the extent of the raster
        minx, miny, maxx, maxy = src.bounds
    # NEED TO ADD A CHECKING CLAUSE FOR THE POLYGONS INTERSECTING THE BBOX
    # Clip the geodataframe to the raster's extent
    clipped_gdf = envelope_gdf.cx[minx:maxx, miny:maxy]
    return clipped_gdf

In [23]:
envelope_list = envelope_gdf.geometry.tolist()
# This function returns 4 lists with the boundary coordinates for all the envelopes that are fully contained inside the raster
def getContainedEnvelopeCoords (raster, envelope_list):
    with rasterio.open(raster, driver='JP2OpenJPEG') as src:
        raster_extent = src.bounds
        
        # List initialization
        minx_list = []
        miny_list = []
        maxx_list = []
        maxy_list = []

        for poly in envelope_list:
            poly_extent = poly.bounds

            # Check if the polygon is fully inside the raster's extent
            if (poly_extent[0] >= raster_extent[0] and poly_extent[2] <= raster_extent[2] and
                poly_extent[1] >= raster_extent[1] and poly_extent[3] <= raster_extent[3]):
                    minx_list.append(poly_extent[0])
                    miny_list.append(poly_extent[1])
                    maxx_list.append(poly_extent[2])
                    maxy_list.append(poly_extent[3])
    return minx_list, miny_list, maxx_list, maxy_list
        

In [71]:
getContainedEnvelopeCoords(path, envelope_list)

([520620.88516150205,
  508887.20875286614,
  520620.88516150205,
  567007.8019472725,
  567007.8019472725,
  567007.8019472725,
  567007.8019472725,
  524047.6961643037,
  524047.6961643037,
  545259.0800683108,
  537844.6872455521,
  531954.8913185342,
  533342.9367736209,
  561467.27803387,
  561467.27803387,
  567007.8019472725,
  574048.1890862663,
  574048.1890862663,
  571197.1618716692,
  597613.292462825,
  589219.6919495184,
  591235.3631517714,
  594549.8695369575,
  588849.6587444162],
 [5632905.883139743,
  5636577.684090835,
  5632905.883139743,
  5670414.32356202,
  5670414.32356202,
  5670414.32356202,
  5670414.32356202,
  5649604.107434307,
  5649604.107434307,
  5631235.007194815,
  5677500.639855889,
  5688577.367889653,
  5657080.622760171,
  5648096.4984890465,
  5648096.4984890465,
  5670414.32356202,
  5666801.50816187,
  5666801.50816187,
  5622269.831121929,
  5593063.380033754,
  5665207.792951369,
  5618893.456633982,
  5691260.201474799,
  5685591.159687021

In [5]:
def export_image(raster, output_path, minx, miny, maxx, maxy):
    # open the raster file (Single Band)
    data = rasterio.open(raster, driver='JP2OpenJPEG')

    # Create a bounding box from the polygon min-max coordinates    
    bbox = box(minx, miny, maxx, maxy)
    # Create a geodataframe with a single polygon so that it can be used with rasterio
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs='32632')
    # Transform the geodataframe to a GeoJSON-like object that can be used as an input in the rasterio mask function
    coords = getFeatures(geo)
    #print(coords)
    
    # Mask and crop the raster AOI where polygon overlaps the whole raster
    out_img, out_transform = mask(data, shapes=coords, crop=True)
    # Define resolution and more
    out_profile = data.profile.copy()
    
    out_profile.update({'width': out_img.shape[2],'height': out_img.shape[1], 'transform': out_transform})
    # Write the extracted raster patch to a file
    with rasterio.open(output_path, 'w', **out_profile) as dst:
        dst.write(out_img)
    
    data.close()
    data = None

## Getting a list of all the band rasters

In [6]:
# Make a list of all .jp2 files
# Maybe this has an error. will check later

dirr = r'C:\Users\Kostas\Downloads\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE'
# This will provide all .jp2 files in every subfolder of the dirr
#jp2_files = glob.glob(dirr+"/**/*.jp2", recursive=True) 

# This will do it for just the bands 01-12. Other products are omitted
jp2_files = glob.glob(dirr+'/**/IMG_DATA/**/R??m/*B??_??m.jp2', recursive=True)
print(jp2_files)

['C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B02_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B03_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B04_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R10m\\L2A_T32UNB_20170420T103021_B08_10m.jp2', 'C:\\Users\\Kostas\\Downloads\\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\\GRANULE\\L2A_T32UNB_A009543_20170420T103454\\IMG_DATA\\R20m\\L2A_T32UNB_20170420T103021_B02_20m.jp2',

## Patch extraction. One band only.

In [ ]:
clipped_gdf = clipGdfToRasterExtent(path, envelope_gdf)
#clipped_gdf.plot()

Using string methods for naming the patches

In [79]:
# Using the split() method. It splits the string wherever an underscore appears and then accesses the second-to-last element.

# Consider adding a checking clause to check if it is in b02, b03 etc....
def imageNaming(raster_path):
    string_parts = raster_path.split("_")
    band_string = string_parts[-3]+ "_" + string_parts[-2]
    return band_string

In [ ]:
# CORRECT
for i in range(0, len(jp2_files)):
    string_parts = jp2_files[i].split("_")
    band_string = string_parts[-3]+ "_" + string_parts[-2]+ "_" + string_parts[-1]
    print(band_string)

In [54]:
for i in range(0, len(jp2_files)):
        string_parts = jp2_files[i].split("_")
        band_string = string_parts[-3] + "_" + string_parts[-2] + "_" + string_parts[-1]
        print(band_string)

20170420T103021_B02_10m.jp2
20170420T103021_B03_10m.jp2
20170420T103021_B04_10m.jp2
20170420T103021_B08_10m.jp2
20170420T103021_B02_20m.jp2
20170420T103021_B03_20m.jp2
20170420T103021_B04_20m.jp2
20170420T103021_B05_20m.jp2
20170420T103021_B06_20m.jp2
20170420T103021_B07_20m.jp2
20170420T103021_B11_20m.jp2
20170420T103021_B12_20m.jp2
20170420T103021_B8A_20m.jp2
20170420T103021_B01_60m.jp2
20170420T103021_B02_60m.jp2
20170420T103021_B03_60m.jp2
20170420T103021_B04_60m.jp2
20170420T103021_B05_60m.jp2
20170420T103021_B06_60m.jp2
20170420T103021_B07_60m.jp2
20170420T103021_B09_60m.jp2
20170420T103021_B11_60m.jp2
20170420T103021_B12_60m.jp2
20170420T103021_B8A_60m.jp2


In [21]:
output_path = r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\pep725_high_count_days\rasters'

# Get the boundaries in minx, miny, maxx, maxy and convert them to a list
# bounds = clipped_gdf.geometry.apply(lambda x : x.bounds).tolist()

NameError: name 'clipped_gdf' is not defined

In [85]:
# Need to add a print statement that shows how many polygons are contained

# Read each raster file from the jp2_files list
for raster in range(0, len(jp2_files)):

    # Get the 4 lists of the minx, miny, maxx, maxy for the polygons fully contained in the raster
    minx_list, miny_list, maxx_list, maxy_list = getContainedEnvelopeCoords(jp2_files[raster], envelope_list)
    
    # Get just the name of the raster (Date_Time_Band) to use it in naming the extracted images
    raster_name = imageNaming(jp2_files[raster])
    print("Creating images around PEP725 stations for the file:", raster_name + ".jp2")

    # Now, for the raster at hand, iterate over each polygon and call the export_image function to extract a patch for each polygon
    for i in range(0, len(minx_list)):
        # Name the patch
        output_file_name = os.path.join(output_path, raster_name + f"_Patch_{i+1}.jp2")
        # Extract it
        export_image(jp2_files[raster], output_file_name, minx_list[i], miny_list[i], maxx_list[i], maxy_list[i])

Polygons to be extracted: 454 

Creating images around PEP725 stations for the file: 20170420T103021_B02.jp2
